In [25]:
#Setup for Sympy
import sympy as sp
import numpy as np

So far we have seen Trapezoidal Rule and Simpson Rule. In general, we may approximate the integral $$I = \int_a^b w(x) f(x) dx$$
where $w \in C^0([a,b]), w>0$ and $w \in L^1((a,b))$ is a weight function, and $f \in L^1((a,b),w)$, by approximating $f$ as a polynomial. 

We may approximate $f$ with $n$-th order Largrange polynomial (sampling points $(x_i)_{i=0}^{k+1}$):
\begin{equation}
f(x) \approx \sum_{i=0}^n f(x_i) \ell_i(x) \quad \ell_i(x) = \prod_{0\leq j \leq i, \, j \neq i} \frac{x-x_j}{x_i-x_j}
\end{equation}

Then
\begin{equation}
\int_a^b w(x)f(x)dx \approx I_n = \sum_{i=0}^n f(x_i) \left(\int_a^b w(x) \ell_i(x) dx \right) = \sum_{i=0}^n w_i f(x_i) 
\end{equation}

We want to choose $(x_i)_{i=0}^n$ such that $I_n(f) = I(f)$ for $f$ polynomial of degree at most $m$, $m>n$ is maximised.

Turns out that if we choose roots of $\phi_{n+1}(x)$ (with $n+1$ degree), so that it belongs to the orthogonal basis of $L^2((a,b))$ (w.r.t. $\langle f,g \rangle = \int_a^b w(x) f(x)g(x) dx$, then $I_n(f) = I(f)$, $f$ any polynomial of degree at most $2n+1$. <br> 
Moreover, we can check by substituting $f = (\ell_i)^2$ that $w_i > 0$. We can find $w_i$ by such substitution, but this is usually tedious. Instead, we find $w_i$ by substituting $f = 1, x, x^2, ..., x_n$ and solve the simultaneous equation.

# Example 1
Let $f \in L^2([-1,1])$ and $w(x) = 1+4|x|$. Construct $I_2(f)$. 

1. Construct Orthogonal Basis: Recall the following formula:
$$\phi_j(x) = \bigg(x - \frac{\langle x\phi_{j-1}(x),\phi_{j-1}(x) \rangle}{\|\phi_{j-1}(x)\|^2} \bigg)\phi_{j-1}(x) - \frac{\|\phi_{j-1}(x)\|^2}{\|\phi_{j-2}(x)\|^2}\phi_{j-2}(x), j \geq 1$$
where $\phi_{-1}(x)=0$ and $\phi_{0}(x)=1$.

In [2]:
# Define Symbols for inner product and norm
x = sp.symbols('x')
w = sp.Function('w')
f = sp.Function('f')
g = sp.Function('g')

In [3]:
# inner product and norms
def inner(w,a,b,f,g):
    output = sp.integrate(w*f*g, (x,a,b))
    return output

def norm(w,a,b,f):
    return sp.sqrt(inner(w,a,b,f,f))

In [4]:
# Define symbols for basis
phi = sp.Function('phi')
phiminus1 = sp.Function('phiminus1')
phiminus2 = sp.Function('phiminus2')

In [5]:
# Iteration
def ortho(w,a,b,r):
    phiminus2 = 1
    phiminus1 = x - inner(w,a,b,x,1)/(norm(w,a,b,1))**2
    if r == 0:
        return [phiminus2]
    elif r == 1:
        return [phiminus2,phiminus1]
    else:
        philist = [phiminus2,phiminus1]
        for i in range(r-1):
            phi = (x - inner(w,a,b,x*phiminus1,phiminus1)/(norm(w,a,b,phiminus1))**2)*phiminus1 - ((norm(w,a,b,phiminus1)/norm(w,a,b,phiminus2))**2)*phiminus2
            phi = sp.simplify(phi)
            philist.append(phi)
            phiminus2 = phiminus1
            phiminus1 = phi
    return philist

In [21]:
# Compute the basis, up to order 2+1 = 3
olist = ortho(1+4*sp.Abs(x),-1,1,3) 
olist

[1, x, x**2 - 4/9, x*(x**2 - 13/20)]

2. Compute the roots of $\phi_3(x)$, as $x_0, x_1, x_2$.

In [31]:
rootlist = sp.solve(olist[3],x)
rootlist

[0, -sqrt(65)/10, sqrt(65)/10]

3. Compute $b_0 = I(1), \, b_1 = I(x), \, b_2 = I(x^2)$

In [23]:
w = 1+4*sp.Abs(x)
b0 = sp.integrate(w,(x,-1,1))
b1 = sp.integrate(x*w,(x,-1,1))
b2 = sp.integrate((x**2)*w,(x,-1,1))
b = sp.Matrix([b0,b1,b2])
b

Matrix([
[  6],
[  0],
[8/3]])

4. Compute $w_0, w_1, w_2$ by solving
\begin{equation}
\begin{cases}
w_0 + w_1 + w_2 = b_0 \\
w_0 x_0 + w_1 x_1 + w_2 x_2 = b_1 \\
w_0 x_0^2 + w_1 x_1^2 + w_2 x_2^2 = b_2
\end{cases}
\end{equation}

In [29]:
A = sp.Matrix([[1,1,1],[rootlist[0],rootlist[1],rootlist[2]], [rootlist[0]**2,rootlist[1]**2,rootlist[2]**2]])
A

Matrix([
[1,            1,           1],
[0, -sqrt(65)/10, sqrt(65)/10],
[0,        13/20,       13/20]])

In [30]:
w0, w1, w2 = sp.symbols('w0,w1,w2')
sp.linsolve((A,b),[w0,w1,w2])

FiniteSet((74/39, 80/39, 80/39))

# Please edit below

In [16]:
w = 1+x**2
a = 0
b = 2
n = 0

In [17]:
# Compute the basis, up to order n+1
olist = ortho(w,a,b,n+1) 
olist

[1, x - 9/7]

In [18]:
# Compute root of phi(n+1)
rootlist = sp.solve(olist[1],x)
rootlist

[9/7]

In [19]:
# Compute actual value of I(f) for f = 1, x,...
b0 = sp.integrate(w,(x,0,2))
b0

14/3

In [20]:
# Compute weight by solving equation
w0 = b0
w0

14/3